In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key
from api_keys import weather_api_key

In [2]:
total_2020=pd.read_csv("../cleaned_csvs/cle_flights_2020.csv", parse_dates=["FL_DATE"])
data_2020_reduced=total_2020[["FL_DATE", "ORIGIN", "DEST", "DEST_CITY_NAME"]]
data_2020_departures=data_2020_reduced.loc[data_2020_reduced["ORIGIN"]=="CLE"]
data_2020_departures['FL_DATE'] = pd.to_datetime(data_2020_departures['FL_DATE']).dt.date
data_2020_departures

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,FL_DATE,ORIGIN,DEST,DEST_CITY_NAME
15,2020-01-31,CLE,ATL,"Atlanta, GA"
16,2020-01-31,CLE,BNA,"Nashville, TN"
17,2020-01-31,CLE,BNA,"Nashville, TN"
18,2020-01-31,CLE,BWI,"Baltimore, MD"
19,2020-01-31,CLE,BWI,"Baltimore, MD"
...,...,...,...,...
22532,2020-03-11,CLE,ORD,"Chicago, IL"
22534,2020-03-11,CLE,DEN,"Denver, CO"
22536,2020-03-11,CLE,ORD,"Chicago, IL"
22537,2020-03-11,CLE,SFO,"San Francisco, CA"


In [5]:
import time
mean_temp=[]
mean_wind=[]
mean_precip=[]
mean_clouds=[] 

for index, row in data_2020_departures.head(10).iterrows():
    city=row["DEST_CITY_NAME"].split("/",)
    date=row["FL_DATE"]
    response = requests.get(f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=b569a70f43cc4efc9f602629202306&q={city[0]}&format=json&date={date}").json()
    try:
        mean_temp.append(response["data"]["weather"][0]["avgtempF"])
        #mean_wind.append(response["data"]["weather"][0]["hourly"][0]["WindGustMiles"])
        #mean_precip.append(response["data"]["weather"][0]["hourly"][0]["precipInches"])
        #mean_clouds.append(response["data"]["weather"][0]["hourly"][0]["cloudcover"])  
    except:
       pass
    #time.sleep(5)

In [6]:
mean_temp

[]

In [138]:
cle_temp=[]
for index, row in data_2020_departures.head().iterrows():
    date=row["FL_DATE"]
    response = requests.get(f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=b569a70f43cc4efc9f602629202306&q=Cleveland&format=json&date={date}").json()
    cle_temp.append(response["data"]["weather"][0]["avgtempF"])

In [139]:
cle_temp

['44', '36', '33', '34', '28']

In [ ]:
data_total_departures["DEST_TEMP"]=mean_temp

{'data': {'error': [{'msg': 'API key is invalid.'}]}}